# Generate pressure coefficients

Read post-processing Cp config file

In [1]:
from cfdmod.use_cases.pressure.cp_config import CpConfig
import pathlib

post_proc_cfg_path = pathlib.Path("./fixtures/tests/pressure/cp_params.yaml") 
post_proc_cfg = CpConfig.from_file(post_proc_cfg_path)

post_proc_cfg

CpConfig(timestep_range=(10000.0, 20000.0), reference_pressure='average', U_H=0.05, statistics=['min', 'max', 'std', 'avg', 'skewness', 'kurtosis'])

Read simulation config file

In [2]:
from nassu.cfg.model import ConfigScheme

sim_cfg_path = pathlib.Path("./fixtures/tests/simulations/setup_nassu_case_v1.5.yaml")

sim_cfg = ConfigScheme.from_file(sim_cfg_path).load_sim_cfgs()[0]

sim_cfg

[2023-10-05 20:05:24.92] [INFO] Loaded from 'fixtures\tests\simulations\setup_nassu_case_v1.5.yaml' 1 simulations ['test_simulation:000']


SimulationConfigs(sim_id=0, name='test_simulation', parent=None, n_steps=20000, save_path=WindowsPath('tests/validation/results/test_simulation'), report=RuntimeFrequencyConfigsEnd(start_step=0, end_step=0, frequency=1000), data=DataConfigs(instantaneous={'default': MacrsExportConfigs(interval=RuntimeFrequencyConfigsStartEnd(start_step=0, end_step=0, frequency=5000), area_export=AreaModel(start=(0.0, 0.0, 0.0), end=(1.0, 1.0, 1.0), is_abs=False), max_lvl=-1, macrs=['rho', 'u'])}, probes=ProbesConfigs(historic_series={'default': SeriesConfigs(macrs=['rho'], interval=RuntimeFrequencyLevelConfigsDefault(start_step=0, end_step=0, frequency=1, lvl=0), lines={}, csvs={}, points={'static_pressure': PointConfigs(pos=(197.0, 40.0, 15.0))}, bodies={'galpao': BodySeriesConfigs(body_name='galpao', normal_offset=0.0625, element_type='cell')})}, spectrum_analysis=SpectrumAnalysisConfigs(macrs=[], points={})), export_IBM_nodes=ExportIBMNodesConfigs(start_step=0, end_step=0, frequency=0), divergence=D

Read Mesh

In [3]:
from nassu.lnas import LagrangianFormat

body_cfg = sim_cfg.domain.bodies["galpao"]

mesh = LagrangianFormat.from_folder(body_cfg.lagrangian_path)

mesh.name, mesh.geometry.vertices.shape, mesh.geometry.triangles.shape

('galpao', (1549, 3), (2915, 3))

Read reference static pressure

In [4]:
from nassu.cfg.schemes.simul import SimulationOutput
from nassu.utils import Environment
import pandas as pd

root_path = pathlib.Path("../nassu")

Environment.set_path(pathlib.Path("../nassu/.env"))

sim_output = SimulationOutput.from_sim_cfg(sim_cfg)

data_path = sim_output.series["default"].points["static_pressure"].data_filename
press_data = pd.read_hdf(root_path /data_path)

press_data

[2023-10-05 20:05:25.40] [INFO] Updated '.env' path to WindowsPath('../nassu/.env')


,point_idx,rho,time_step
2915,0,1.000000,1.0
2915,0,1.000000,2.0
2915,0,1.000000,3.0
2915,0,1.000000,4.0
2915,0,1.000000,5.0
...,...,...,...
2915,0,1.002887,19997.0
2915,0,1.002865,19998.0
2915,0,1.002845,19999.0
2915,0,1.002825,20000.0


Read body hist series

In [5]:
body_data_path = sim_output.series["default"].bodies["galpao"].data_filename
body_data = pd.read_hdf(root_path / body_data_path)

body_data

,point_idx,rho,time_step
0,0,1.000000,1.0
1,1,1.000000,1.0
2,2,1.000000,1.0
3,3,1.000000,1.0
4,4,1.000000,1.0
...,...,...,...
2910,2910,1.004250,20001.0
2911,2911,1.004582,20001.0
2912,2912,1.004572,20001.0
2913,2913,1.004094,20001.0


Slice data from post processing specs

In [6]:
press_data_use = press_data.loc[(press_data["time_step"] >= post_proc_cfg.timestep_range[0]) & (press_data["time_step"] <= post_proc_cfg.timestep_range[1])]
body_data_use = body_data.loc[(body_data["time_step"] >= post_proc_cfg.timestep_range[0]) & (body_data["time_step"] <= post_proc_cfg.timestep_range[1])]

press_data_use.set_index('time_step', inplace=True)
body_data_use.set_index('time_step', inplace=True)

Cp transformation

In [7]:
average_static_pressure = press_data_use['rho'].to_numpy().mean()
dynamic_pressure = 0.5 * average_static_pressure * post_proc_cfg.U_H ** 2
cs_square = 1 / 3

# Broadcast subtraction operation
transformed_data = body_data_use.copy().drop(columns=["rho"])

if post_proc_cfg.reference_pressure == "instantaneous":
    transformed_data["cp"] = cs_square * (body_data_use["rho"] - body_data_use.index.map(press_data_use['rho'])) / dynamic_pressure
elif post_proc_cfg.reference_pressure == "average":
    transformed_data["cp"] = cs_square * (body_data_use["rho"] - average_static_pressure) / dynamic_pressure

transformed_data.reset_index(inplace=True)

# OUTPUT 1: cp(t)
transformed_data

,time_step,point_idx,cp
0,10000.0,0,0.268273
1,10000.0,1,0.273546
2,10000.0,2,0.265319
3,10000.0,3,0.276913
4,10000.0,4,0.272593
...,...,...,...
29152910,20000.0,2910,0.923266
29152911,20000.0,2911,1.014458
29152912,20000.0,2912,1.011409
29152913,20000.0,2913,0.880417


Calculate pressure coefficient statistics

In [8]:
group_by_point = transformed_data.groupby('point_idx')

statistics_data = pd.DataFrame({'point_idx': transformed_data['point_idx'].unique()})

if "avg" in post_proc_cfg.statistics:
    statistics_data['cp_avg'] = group_by_point['cp'].mean()
if "min" in post_proc_cfg.statistics:
    statistics_data['cp_min'] = group_by_point['cp'].min()
if "max" in post_proc_cfg.statistics:
    statistics_data['cp_max'] = group_by_point['cp'].max()
if "std" in post_proc_cfg.statistics:
    statistics_data['cp_rms'] = group_by_point['cp'].std()

# Calculate skewness and kurtosis using apply
if "skewness" in post_proc_cfg.statistics:
    skewness = group_by_point['cp'].apply(lambda x: x.skew()).reset_index(name='skewness')
    statistics_data['cp_skewness'] = skewness["skewness"]
if "kurtosis" in post_proc_cfg.statistics:
    kurtosis = group_by_point['cp'].apply(lambda x: x.kurt()).reset_index(name='kurtosis')
    statistics_data['cp_kurtosis'] = kurtosis["kurtosis"]

# OUTPUT 2: cp_stats
statistics_data

,point_idx,cp_avg,cp_min,cp_max,cp_rms,cp_skewness,cp_kurtosis
0,0,-0.021076,-2.392591,2.145996,0.975655,0.007537,-0.644743
1,1,-0.019863,-2.389510,2.147012,0.974600,0.005196,-0.641696
2,2,-0.021279,-2.394671,2.139135,0.975093,0.006027,-0.645222
3,3,-0.019417,-2.387731,2.152889,0.975021,0.005858,-0.640871
4,4,-0.020308,-2.389669,2.148188,0.974958,0.006443,-0.642380
...,...,...,...,...,...,...,...
2910,2910,0.065866,-2.437123,2.166579,0.996299,-0.073992,-0.699826
2911,2911,0.025314,-2.499840,2.120172,0.984458,-0.098281,-0.592226
2912,2912,0.032308,-2.501142,2.110929,0.985440,-0.100940,-0.606598
2913,2913,0.078103,-2.352251,2.193546,1.003240,-0.043307,-0.751864


Combine statistics with mesh and create a VTK object

In [9]:
from cfdmod.api.vtk.write_vtk import write_polydata, create_polydata_for_cell_data

output_path = pathlib.Path("./")
vtk_path = output_path / "cp_stats.vtk"

polydata = create_polydata_for_cell_data(data=statistics_data, mesh=mesh.geometry)

# OUTPUT 3: VTK cp_stats
write_polydata(vtk_path, polydata)